In [63]:
import pandas as pd
import io
import requests

# Import traitor data

In [64]:
traitor_df = pd.read_csv("147_traitors.csv", index_col=False)

In [65]:
traitor_df

,id,full_name,first_name,last_name,state,chamber,top_donors_link
0,N00003028,Robert B. Aderholt,Robert,Aderholt,Ala.,House,https://www.opensecrets.org/members-of-congres...
1,N00030910,Mo Brooks,Mo,Brooks,Ala.,House,https://www.opensecrets.org/members-of-congres...
2,NaN,Jerry Carl,Jerry,Carl,Ala.,House,NaN
3,NaN,Barry Moore,Barry,Moore,Ala.,House,NaN
4,N00035691,Gary Palmer,Gary,Palmer,Ala.,House,https://www.opensecrets.org/members-of-congres...
...,...,...,...,...,...,...,...
142,N00034044,John Kennedy,John,Kennedy,La.,Senate,https://www.opensecrets.org/members-of-congres...
143,N00043298,Cindy Hyde-Smith,Cindy,Hyde-Smith,Miss.,Senate,https://www.opensecrets.org/members-of-congres...
144,N00041620,Josh Hawley,Josh,Hawley,Mo.,Senate,https://www.opensecrets.org/members-of-congres...
145,N00033085,Ted Cruz,Ted,Cruz,Texas,Senate,https://www.opensecrets.org/members-of-congres...


# Remove NAN data

In [66]:
traitor_df = traitor_df.fillna("")

In [67]:
traitor_df

,id,full_name,first_name,last_name,state,chamber,top_donors_link
0,N00003028,Robert B. Aderholt,Robert,Aderholt,Ala.,House,https://www.opensecrets.org/members-of-congres...
1,N00030910,Mo Brooks,Mo,Brooks,Ala.,House,https://www.opensecrets.org/members-of-congres...
2,,Jerry Carl,Jerry,Carl,Ala.,House,
3,,Barry Moore,Barry,Moore,Ala.,House,
4,N00035691,Gary Palmer,Gary,Palmer,Ala.,House,https://www.opensecrets.org/members-of-congres...
...,...,...,...,...,...,...,...
142,N00034044,John Kennedy,John,Kennedy,La.,Senate,https://www.opensecrets.org/members-of-congres...
143,N00043298,Cindy Hyde-Smith,Cindy,Hyde-Smith,Miss.,Senate,https://www.opensecrets.org/members-of-congres...
144,N00041620,Josh Hawley,Josh,Hawley,Mo.,Senate,https://www.opensecrets.org/members-of-congres...
145,N00033085,Ted Cruz,Ted,Cruz,Texas,Senate,https://www.opensecrets.org/members-of-congres...


# Download contribution CSV files from Open Secrets for each traitor and combine to master CSV - 147_contributors.csv

In [6]:
# starter dataframe for master contributor data
contributor_column_names = ['pacid','orgid','ultorg','total','pacs','indivs','releasedate','traitor_id']
contributor_df = pd.DataFrame(columns = contributor_column_names)

# loop through all traitors
for _, traitor_row in traitor_df.iterrows():     
    if traitor_row.id != ' ':
        print(f'{traitor_row.full_name} from {traitor_row.state} is a traitor')
        
        # get contributors csv file
        url = f'https://www.opensecrets.org/members-of-congress/{traitor_row.first_name.lower()}-{traitor_row.last_name.lower()}/contributors.csv?cid={traitor_row.id}&cycle=2020&recs=100&type=C'
        data = requests.get(url).content
        df = pd.read_csv(io.StringIO(data.decode('utf-8')))
        
        # remove trivial columns
        df = df.drop(['affiliateid', 'rank', 'client', 'registrant', 'numlobbyists', 'loborgid'], axis=1)
        
        # add traitor data
        df['traitor_id'] = traitor_row.id     
        df['traitor_name'] = traitor_row.full_name
        df['traitor_state'] = traitor_row.state
        df['traitor_chamber'] = traitor_row.chamber
        
        # combine to master contributor file
        frames = [contributor_df, df]
        contributor_df = pd.concat(frames)

# rename columns        
contributor_df = contributor_df.rename(columns={'pacid': 'pac_id', 'orgid': 'id', 'ultorg': 'name', 'indivs': 'individuals', 'releasedate': 'release_date'})  

print('completed')

# save contributors dataframe to csv file
contributor_df.to_csv(r'147_contributors.csv', index=False, header=True)          

Robert B. Aderholt
Mo Brooks
Gary Palmer
Mike Rogers
Andy Biggs
Paul Gosar
Debbie Lesko
David Schweikert
Rick Crawford
Ken Calvert
Mike Garcia
Darrell Issa
Doug LaMalfa
Kevin McCarthy
Devin Nunes
Doug Lamborn
Mario Diaz-Balart
Neal Dunn
Matt Gaetz
Brian Mast
Bill Posey
John Rutherford
Greg Steube
Daniel Webster
Rick Allen
Earl L. "Buddy" Carter
Jody Hice
Barry Loudermilk
Russ Fulcher
Mike Bost
Jim Baird
Jim Banks
Greg Pence
Jackie Walorski
Ron Estes
Harold Rogers
Garret Graves
Clay Higgins
Steve Scalise
Andy Harris
Jack Bergman
Tim Walberg
Jim Hagedorn
Michael Guest
Trent Kelly
Steven Palazzo
Sam Graves
Vicky Hartzler
Billy Long
Blaine Luetkemeyer
Jason Smith
Dan Bishop
Ted Budd
Virginia Foxx
Richard Hudson
Gregory Murphy
David Rouzer
Jeff Van Drew
Chris Jacobs
Elise M. Stefanik
Lee Zeldin
Adrian Smith
Steve Chabot
Warren Davidson
Bob Gibbs
Bill Johnson
Jim Jordan
Tom Cole
Kevin Hern
Frank Lucas
Markwayne Mullin
John Joyce
Fred Keller
Mike Kelly
Daniel Meuser
Scott Perry
Guy Reschentha

In [14]:
import requests
for index, row in df.iterrows():  
    if index < 3:
        if row[5] != ' ':
            url = f'https://www.opensecrets.org/members-of-congress/{row[3].lower()}-{row[4].lower()}/contributors.csv?cid={row[5]}&cycle=2020&recs=100&type=C'
            print(url)
            r = requests.get(url)
            with open(str(row[5]) + '.csv', 'wb') as outfile:
                outfile.write(r.content)

https://www.opensecrets.org/members-of-congress/robert-aderholt/contributors.csv?cid=N00003028&cycle=2020&recs=100&type=C
https://www.opensecrets.org/members-of-congress/mo-brooks/contributors.csv?cid=N00030910&cycle=2020&recs=100&type=C


# Top Contributors to all Traitors

In [61]:
# load contributors data
contributor_df = pd.read_csv("147_contributors.csv", index_col=False)

# group by donation amount
contributor_df_sum = contributor_df.groupby(['name'])['total'].agg('sum').reset_index(name="total_contributions")

# sort by total donation highest to lowest
contributor_df_sum = contributor_df_sum.sort_values(by = ['total_contributions'], ascending=[False])
contributor_df_sum.reset_index(drop=True, inplace=True)

In [62]:
contributor_df_sum

,name,total_contributions
0,American Bankers Assn,1095785
1,National Assn of Realtors,1073620
2,Majority Cmte PAC,1060000
3,House Freedom Fund,1052229
4,AT&T Inc,978173
...,...,...
5302,Jackson Clinic,250
5303,Parks Group,250
5304,Nutrien,50
5305,Visnaga Ranch,50


In [68]:
# save contributors dataframe to csv file
contributor_df_sum.to_csv(r'147_contributors_top_list.csv', index=False, header=True)          